# YOLOv5 Pseudo Labeling

According to the results of [this notebook](https://www.kaggle.com/nvnnghia/fasterrcnn-pseudo-labeling) FaterRCNN seems to work well with Pseudo Labeling.
In this notebook I am going to test Pseudo labeling technique on Yolov5.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import shutil as sh

# Getting yolov5 repo 

In [2]:
#!git clone https://github.com/ultralytics/yolov5
#!mv yolov5/* ./

!cp -r ../input/yolov5-pseudo-labeling/* .

In [3]:
!pip install --no-deps '../input/weightedboxesfusion/' > /dev/null

# Convert train data to yolov5 format
Based on [this notebook](https://www.kaggle.com/orkatz2/yolov5-train)

In [4]:
R_fold = 1
def convertTrainLabel():
    df = pd.read_csv('../input/global-wheat-detection/train.csv')
    bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        df[column] = bboxs[:,i]
    df.drop(columns=['bbox'], inplace=True)
    df['x_center'] = df['x'] + df['w']/2
    df['y_center'] = df['y'] + df['h']/2
    df['classes'] = 0
    from tqdm.auto import tqdm
    import shutil as sh
    df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
    
    index = list(set(df.image_id))
    
    source = 'train'
    if True:
        for fold in [0]:
            val_index = index[len(index)*R_fold//5:len(index)*(R_fold+1)//5]
            for name,mini in tqdm(df.groupby('image_id')):
                if name in val_index:
                    path2save = 'val2017/'
                else:
                    path2save = 'train2017/'
                if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                    os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
                with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                    row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                    row = row/1024
                    row = row.astype(str)
                    for j in range(len(row)):
                        text = ' '.join(row[j])
                        f.write(text)
                        f.write("\n")
                if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                    os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
                sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

# Some useful functions
TTA, WBF, etc

In [5]:
from ensemble_boxes import *
def run_wbf(boxes, scores, image_size=1023, iou_thr=0.5, skip_box_thr=0.7, weights=None):
    #boxes = [prediction[image_index]['boxes'].data.cpu().numpy()/(image_size-1) for prediction in predictions]
    #scores = [prediction[image_index]['scores'].data.cpu().numpy() for prediction in predictions]
    labels = [np.zeros(score.shape[0]) for score in scores]
    boxes = [box/(image_size) for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #boxes, scores, labels = nms(boxes, scores, labels, weights=[1,1,1,1,1], iou_thr=0.5)
    boxes = boxes*(image_size)
    return boxes, scores, labels

def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

def detect1Image(im0, imgsz, model, device, conf_thres, iou_thres):
    img = letterbox(im0, new_shape=imgsz)[0]
    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img =  img.float()  # uint8 to fp16/32
    img /= 255.0   
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    boxes = []
    scores = []
    for i, det in enumerate(pred):  # detections per image
        # save_path = 'draw/' + image_id + '.jpg'
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Write results
            for *xyxy, conf, cls in det:
                boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                scores.append(conf)

    return np.array(boxes), np.array(scores) 

# Make pseudo labels for Yolov5

In [6]:
from utils.datasets import *
from utils.utils import *

def makePseudolabel(weights = '../input/yolov5pth/weightsbest_yolov5x_fold3.pt'):
    source = '../input/global-wheat-detection/test/'
#     weights = '../input/yolov5pth/weightsbest_yolov5x_fold3.pt'
    imgsz = 1024
    conf_thres = 0.5
    iou_thres = 0.6
    is_TTA = True
    
    imagenames =  os.listdir(source)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load model
    model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
    model.to(device).eval()
    
    dataset = LoadImages(source, img_size=imgsz)

    path2save = 'train2017/'
    if not os.path.exists('convertor/fold0/labels/'+path2save):
        os.makedirs('convertor/fold0/labels/'+path2save)
    if not os.path.exists('convertor/fold0/images/{}'.format(path2save)):
        os.makedirs('convertor/fold0/images/{}'.format(path2save))
            
    for name in tqdm(imagenames):
        image_id = name.split('.')[0]
        im01 = cv2.imread('%s/%s.jpg'%(source,image_id))  # BGR
#         im01 = cv2.resize(im01,(1024,1024))
        if im01.shape[0]!=1024 or im01.shape[1]!=1024:
            continue
        assert im01 is not None, 'Image Not Found '
        # Padded resize
        im_w, im_h = im01.shape[:2]
        if is_TTA:
            enboxes = []
            enscores = []
            for i in range(4):
                im0 = TTAImage(im01, i)
                boxes, scores = detect1Image(im0, imgsz, model, device, conf_thres, iou_thres)
                for _ in range(3-i):
                    boxes = rotBoxes90(boxes, im_w, im_h)
                    
                enboxes.append(boxes)
                enscores.append(scores) 

            boxes, scores, labels = run_wbf(enboxes, enscores, image_size = im_w, iou_thr=0.6, skip_box_thr=0.43)
            boxes = boxes.astype(np.int32).clip(min=0, max=im_w)
        else:
            boxes, scores = detect1Image(im01, imgsz, model, device, conf_thres, iou_thres)

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        boxes = boxes[scores >= 0.1].astype(np.int32)
        scores = scores[scores >=float(0.1)]
        
        lineo = ''
        for box in boxes:
            x1, y1, w, h = box
            xc, yc, w, h = (x1+w/2)/1024, (y1+h/2)/1024, w/1024, h/1024
            lineo += '0 %f %f %f %f\n'%(xc, yc, w, h)
            
        fileo = open('convertor/fold0/labels/'+path2save+image_id+".txt", 'w+')
        fileo.write(lineo)
        fileo.close()
        sh.copy("../input/global-wheat-detection/test/{}.jpg".format(image_id),'convertor/fold0/images/{}/{}.jpg'.format(path2save,image_id))

In [7]:
convertTrainLabel()

In [8]:
makePseudolabel()

FileNotFoundError: [Errno 2] No such file or directory: '../input/yolov5pth/weightsbest_yolov5x_fold3.pt'

# Retrain yolov5 with pseudo data

In [9]:
if len(os.listdir('../input/global-wheat-detection/test/'))<11:
    pass
#     !python train.py --img 1024 --batch 4 --epochs 1 --data ../input/configyolo5/wheat0.yaml --cfg ../input/newcon/yolov5x3.yaml --weights ../input/yolov5pth/weightsbest_yolov5x_fold3.pt  
else:
    !python train.py --img 1024 --batch 4 --epochs 10 --data ../input/configyolo5/wheat0.yaml --cfg ../input/newcon/yolov5x3.yaml --weights ../input/yolov5pth/weightsbest_yolov5x_fold3.pt    
!rm -rf convertor

In [10]:
# if not os.path.exists('weights/best.pt'):
#     weights = '../input/yolov5pth/best_yolov5x_fold3.pt'
# else:
#     convertTrainLabel()
#     makePseudolabel(weights = 'weights/best.pt')
#     if len(os.listdir('../input/global-wheat-detection/test/'))<11:
#         !python train.py --img 1024 --batch 4 --epochs 1 --data ../input/configyolo5/wheat0.yaml --cfg ../input/newcon/yolov5x3.yaml --weights weights/best.pt 
#     else:
#         !python train.py --img 1024 --batch 4 --epochs 4 --data ../input/configyolo5/wheat0.yaml --cfg ../input/newcon/yolov5x3.yaml --weights weights/best.pt  
#     !rm -rf convertor

In [11]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

# Final prediction

In [12]:
# !cp ../input/yolo5tta4/yolo.py ./models/

In [13]:
from utils.datasets import *
from utils.utils import *

In [14]:
def detect1Image_aug(im0, imgsz, model, device, conf_thres, iou_thres):
    img = letterbox(im0, new_shape=imgsz)[0]
    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img =  img.float()  # uint8 to fp16/32
    img /= 255.0   
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    boxes = []
    scores = []
    for i, det in enumerate(pred):  # detections per image
        # save_path = 'draw/' + image_id + '.jpg'
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Write results
            for *xyxy, conf, cls in det:
                boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                scores.append(conf)

    return np.array(boxes), np.array(scores) 

In [15]:
def clip_coords2(boxes, img_shape):
    # Clip bounding xyxy bounding boxes to image shape (height, width)
    boxes[:, 0].clamp_(0, img_shape[1])  # x1
    boxes[:, 1].clamp_(0, img_shape[0])  # y1
    boxes[:, 2].clamp_(0, img_shape[1])  # x2
    boxes[:, 3].clamp_(0, img_shape[0])  # y2
    
def scale_coords2(coords,factorx,factory, img0_shape):
    coords[:, 0::2] *= factorx  # x padding
    coords[:, 1::2] *= factory  # y padding
    clip_coords2(coords, img0_shape)
    return coords

In [16]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [17]:
!pip install /kaggle/input/orkatzfdata/yacs-0.1.7-py3-none-any.whl
!mkdir fvcore
!cp -R '/kaggle/input/orkatzfdata/fvcore-0.1.dev200407/fvcore-0.1.dev200407/' ./fvcore
!pip install fvcore/fvcore-0.1.dev200407/.
!mkdir detectron2-ResNeSt
!cp -R /kaggle/input/orkatzfdata/detectron2-ResNeSt/* ./detectron2-ResNeSt/
!pip install detectron2-ResNeSt/.

Processing /kaggle/input/orkatzfdata/yacs-0.1.7-py3-none-any.whl
Processing ./fvcore/fvcore-0.1.dev200407
  Created wheel for fvcore: filename=fvcore-0.1.dev200407-py3-none-any.whl size=38762 sha256=f88ee7553248fc7929ebdc9ba6abc698d826db01f0d1ba8ad95f50e18d75020d
  Stored in directory: /root/.cache/pip/wheels/1e/e5/f3/be414e6471636ec89b15d85636ec76836ce2fbacb74d98c7e3
Successfully built fvcore
Processing ./detectron2-ResNeSt
  Created wheel for detectron2: filename=detectron2-0.1.1-cp37-cp37m-linux_x86_64.whl size=4872812 sha256=f8f767245bf83781bc9c922fa7508c9b69e8b63b09037ffecb320256a5fa4461
  Stored in directory: /root/.cache/pip/wheels/0f/f3/e1/8a8e7f0a7e7fcc4215726cbd3a05188930dbe6fe04dc585379
Successfully built detectron2


In [18]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
import os
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
from ensemble_boxes import *
import torch
import random
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from tqdm import tqdm
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet
from sklearn.model_selection import StratifiedKFold
from skopt import gp_minimize, forest_minimize
from skopt.utils import use_named_args
from skopt.plots import plot_objective, plot_evaluations, plot_convergence, plot_regret
from skopt.space import Categorical, Integer, Real

In [19]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint)

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

def load_net7(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d7')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 1024
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    net = torch.nn.DataParallel(net)
    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

In [20]:
models2 = [load_net7('../input/tempb7/best-checkpoint-015epoch.bin'),
           load_net7('../input/tempb7/best-checkpoint-020epoch.bin'),
           load_net7('../input/tempb7/best-checkpoint-022epoch.bin'),]
models = [
        load_net('../input/effdetbestpth/best-fold0-augmix.pth'),
        load_net('../input/effdetbestpth/best-fold3.pth'),
        load_net('../input/effdetbestpth/best-fold4.pth'),
        load_net('../input/effdetbestpth/best-fold1-augmix.pth'),
        load_net('../input/effdetbestpth/best-fold2-augmix.pth'),
        load_net('../input/effdetbestpth/fold0-best.bin')
]

FileNotFoundError: [Errno 2] No such file or directory: '../input/tempb7/best-checkpoint-015epoch.bin'

In [21]:
from detectron2.config import get_cfg
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_cascade_rcnn_ResNeSt_101_FPN_syncbn_range-scale_1x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 


cfg.MODEL.WEIGHTS = os.path.join('/kaggle/input/best-inrae-1/', "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.47  # set the testing threshold for this model
cfg.DATASETS.TEST = ("m5_val", )
predictor1 = DefaultPredictor(cfg)

In [22]:
models3 =[predictor1]

In [23]:
DATA_ROOT_PATH = '../input/global-wheat-detection/test'

class TestDatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None,transforms2=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms
        self.transforms2 = transforms2

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image0 = sample['image']
        if self.transforms2:
            sample = {'image': image}
            sample = self.transforms2(**sample)
            image1 = sample['image']
        return image0,image1, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
    )

def get_valid_transforms2():
    return A.Compose(
        [
            A.Resize(height=1024, width=1024, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
    )

dataset = TestDatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms(),transforms2=get_valid_transforms2()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [24]:
def make_predictions(
    images, images1,image_ids,
    score_threshold=0.25,
):
    images = images.cuda().float()
    images1 = images1.cuda().float()
    image_id = image_ids
    rh,rw,_ = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg').shape
    Hscale512 = rh/512
    Wscale512 = rw/512
    
    Hscale1024 = rh/1024
    Wscale1024 = rw/1024
    predictions = []
    for fold_number, net in enumerate(models):
        with torch.no_grad():
            det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
            result = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
#                 boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                
                boxes[:, 0::2] *= Wscale512
                boxes[:, 1::2] *= Hscale512
                result.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                })
            predictions.append(result)
    
    for fold_number, net in enumerate(models2):
        with torch.no_grad():
            det = net(images1, torch.tensor([1]*images1.shape[0]).float().cuda())
            result = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
#                 boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                boxes[:, 0::2] *= Wscale1024
                boxes[:, 1::2] *= Hscale1024
                result.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                })
            predictions.append(result)
    for fold_number, net in enumerate(models3):
        image_id = image_ids
        result =[]
        with torch.no_grad():
            im = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
            outputs = predictor1(im)
            out = outputs["instances"].to("cpu")
            scores = out.get_fields()['scores'].numpy()
            boxes = out.get_fields()['pred_boxes'].tensor.numpy().astype(int)
            labels= out.get_fields()['scores'].numpy()
            result.append({
                    'boxes': boxes,
                    'scores': scores,
                })
            predictions.append(result)
            predictions.append(result)
            
    return predictions

In [25]:
def clip_coords3(boxes, img_shape):
    # Clip bounding xyxy bounding boxes to image shape (height, width)
    boxes[:, 0].clamp_(0, img_shape[1])  # x1
    boxes[:, 1].clamp_(0, img_shape[0])  # y1
    boxes[:, 2].clamp_(0, img_shape[1])  # x2
    boxes[:, 3].clamp_(0, img_shape[0])  # y2
    return boxes

In [26]:
def run_wbf2(predictions, image_index, image_size=1024, iou_thr=0.34, skip_box_thr=0.33, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]#+ [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions1]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]#+[prediction[image_index]['scores'].tolist()  for prediction in predictions1]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]#+[np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions1]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [27]:
def detect():
    transforms = get_valid_transforms()
    transforms2 = get_valid_transforms2()
    source = '../input/global-wheat-detection/test/'
    weights = 'weights/best.pt'
    weights800 = '../input/yolo800/best_yolov5x_fold0_800.pt'
    if not os.path.exists(weights):
        weights = '../input/yolov5pth/weightsbest_yolov5x_fold3.pt'
    imgsz = 1024
    conf_thres = 0.34
    iou_thres = 0.45
    is_TTA = True
    
    imagenames =  os.listdir(source)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load model
    model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
    model.to(device).eval()
    
    model800 = torch.load(weights800, map_location=device)['model'].float()  # load to FP32
    model800.to(device).eval()
    
    dataset = LoadImages(source, img_size=imgsz)

    results = []
    fig, ax = plt.subplots(5, 2, figsize=(30, 70))
    count = 0
    # img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    #for path, img, im0s, _ in dataset:
    for name in tqdm(imagenames):
        image_id = name.split('.')[0]
        im01 = cv2.imread('%s/%s.jpg'%(source,image_id))  # BGR
        image = cv2.cvtColor(im01, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        sample = {'image': image}
        sample = transforms(**sample)
        image0 = sample['image'] #512

        sample = {'image': image}
        sample = transforms2(**sample)
        image1 = sample['image'] #1024
        
        if image1.ndimension() == 3:
            image1 = image1.unsqueeze(0)
            image0 = image0.unsqueeze(0)
        assert im01 is not None, 'Image Not Found '
        # Padded resize
        im_w, im_h = im01.shape[:2]
        if is_TTA:
            enboxes = []
            enscores = []
            for i in range(4):
                im0 = TTAImage(im01, i)
                boxes, scores = detect1Image(im0, imgsz, model, device, conf_thres, iou_thres)
                for _ in range(3-i):
                    boxes = rotBoxes90(boxes, im_w, im_h)
                    
                if 1: #i<3:
                    enboxes.append(boxes)
                    enscores.append(scores) 
                    
            boxes, scores = detect1Image(im01, imgsz, model, device, conf_thres, iou_thres)
            enboxes.append(boxes)
            enscores.append(scores)
            enboxes.append(boxes)
            enscores.append(scores)
            
#             boxes, scores = detect1Image_aug(im01, 800, model800, device, 0.1, 0.94)
#             boxes, scores, labels = run_wbf([boxes], [scores], image_size = 800, iou_thr=0.4, skip_box_thr=0.3)
#             enboxes.append(boxes)
#             enscores.append(scores)


            
            predictions = make_predictions(image0,image1,image_id)
            boxes, scores, labels = run_wbf2(predictions, image_index=0)
            
            enboxes.append(boxes)
            enscores.append(scores)
            enboxes.append(boxes)
            enscores.append(scores)
            
            
            boxes, scores, labels = run_wbf(enboxes, enscores, image_size = im_w, iou_thr=0.48, skip_box_thr=0.575)
            boxes = boxes.astype(np.int32).clip(min=0, max=im_w)
        else:
            boxes, scores = detect1Image(im01, imgsz, model, device, conf_thres, iou_thres)

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        boxes = boxes[scores >= 0.05].astype(np.int32)
        scores = scores[scores >=float(0.05)]
        if count<10:
            #sample = image.permute(1,2,0).cpu().numpy()
            for box, score in zip(boxes,scores):
                cv2.rectangle(im0,
                              (box[0], box[1]),
                              (box[2]+box[0], box[3]+box[1]),
                              (220, 0, 0), 2)
                cv2.putText(im0, '%.2f'%(score), (box[0]+np.random.randint(20), box[1]), cv2.FONT_HERSHEY_SIMPLEX ,  
                   0.5, (255,255,255), 2, cv2.LINE_AA)
            ax[count%5][count//5].imshow(im0)
            count+=1
            
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }

        results.append(result)
    return results

In [28]:
results = detect()
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/yolov5pth/weightsbest_yolov5x_fold3.pt'

In [29]:
!ls weights/

download_weights.sh
